In [1]:
import numpy as np
from abc import ABC, abstractmethod
import ipywidgets as widgets

# Abstract signal class to hold all common signal methods
class AbstractSignal(ABC):
    interval = 0.01
    limit = 4
    x = np.arange(-limit, limit, interval)

    def __init__(self, amplitude=1, width=1, delay=0, t=0, frequency=1, phase=0):
        self.amplitude = amplitude
        self.width = width
        self.delay = delay
        self.t = t
        self.frequency = frequency
        self.phase = phase
        
    @abstractmethod
    def points(self):
        pass
    
    # Add padding zeros before and after the signal
    def buffer_y(self, i, vals):
        buffer = np.zeros(len(AbstractSignal.x))
        return np.concatenate((buffer[:i], vals, buffer[i:]))[:len(AbstractSignal.x)]
    
    # Create the x and y data for the plot
    def generate_curve(self, flip=False):
        i = np.abs(AbstractSignal.x - self.delay).argmin()
        y = self.buffer_y(i, self.points)
        if flip:
            y = np.flip(y, 0)
            
        if self.t != 0:
            s = np.abs(AbstractSignal.x - self.t).argmin() - int(len(AbstractSignal.x)/2)
            if s < 0:
                y = np.concatenate((y, np.zeros(abs(s))))[-len(AbstractSignal.x):]
            else:
                y = np.concatenate((np.zeros(abs(s)), y))[:len(AbstractSignal.x)]
    
        return AbstractSignal.x, y
    
    # Convolve the current function with the provided function
    def convolve(self, other_signal):
        c = np.convolve(self.points, other_signal.points)
        if not(len(self.points) < 2 or len(other_signal.points) < 2):
            c *= AbstractSignal.interval
        i = np.abs(AbstractSignal.x - (self.delay + other_signal.delay)).argmin()
        y = self.buffer_y(i, c)
        return AbstractSignal.x, y
    
    # Update internal params on slider event
    def update(self, params):
        self.amplitude = params.get("Amplitude", self.amplitude)
        self.width = params.get("Width", self.width)
        self.delay = params.get("Delay", self.delay)
        self.t = params.get("t", self.t)
        self.frequency = params.get("Frequency", self.frequency)
        self.phase = params.get("Phase", self.phase)
        
# Defines a step signal
class Step(AbstractSignal):
    @property
    def points(self):
        return self.amplitude * np.heaviside(np.arange(0, AbstractSignal.limit * 2, AbstractSignal.interval), 1)

# Defines a pulse signal
class Pulse(AbstractSignal):       
    @property
    def points(self):
        return self.amplitude * np.heaviside(np.arange(0, self.width, AbstractSignal.interval), 1)
    
# Defines a impulse signal
class Impulse(AbstractSignal):       
    @property
    def points(self):
        return self.amplitude * np.array([1])
    
# Defines a cosine signal
class Cosine(AbstractSignal):       
    @property
    def points(self):
        return self.amplitude * np.cos(self.frequency * 2 * np.pi * (np.arange(0,
                                    self.width + AbstractSignal.interval, AbstractSignal.interval) - self.phase) / self.width)

# Defines a triangle signal 
class Triangle(AbstractSignal):       
    @property
    def points(self):
        return self.amplitude * np.concatenate((np.arange(0, self.width, AbstractSignal.interval*2),
                                               self.width-np.arange(0, self.width, AbstractSignal.interval*2))) / self.width


In [2]:
# Slider class to combine slide and play handler
class Slider:
    def __init__(self, call_back, descp, default_val=0, minimum=0, maximum=10):
        self.player = widgets.Play(min=minimum, interval=500, value=default_val)
        self.slider = widgets.FloatSlider(description=descp, min=minimum, max=maximum, step=0.1, continuous_update=True)
        self.link(call_back)
  
    def link(self, on_value_change):
        self.slider.observe(on_value_change, 'value')
        widgets.jslink((self.player, 'value'), (self.slider, 'value'))
        
    def get_display(self):
        return widgets.HBox([self.slider, self.player])

In [3]:
import matplotlib.pyplot as plt

class Signal:
    def __init__(self):
        self.f_sliders = [Slider(self.f_call_back, *s) for s in
                        [['Amplitude', 1, -4, 4], ['Width', 1, 0.1, 2], ['Delay', 0, -2, 2], ['Frequency', 1, 0, 4], ['Phase', 0, -2, 2]]]
        self.g_sliders = [Slider(self.g_call_back, *s) for s in
                        [['Amplitude', 1, -4, 4], ['Width', 1, 0.1, 2], ['Delay', 0, -2, 2], ['Frequency', 1, 0, 4], ['Phase', 0, -2, 2]]]
        self.t_slider = Slider(self.t, 't', 0, -2, 2)
        
        self.f_dropdown = widgets.Dropdown(options=['Step', 'Pulse', 'Impulse', 'Triangle', 'Cosine'],
                                         value='Pulse', description=r'f($\tau$) =', disabled=False)
        self.g_dropdown = widgets.Dropdown(options=['Step', 'Pulse', 'Impulse', 'Triangle', 'Cosine'],
                                         value='Pulse', description=r'g($\tau$) =', disabled=False)
        
        self.f_dropdown.observe(self.f_on_value_change, 'value')
        self.g_dropdown.observe(self.g_on_value_change, 'value')
        
        self.f = Pulse()
        self.g = Pulse()
        
        # Setup the plots
        self.out = widgets.Output()
        self.fig, (self.ax, self.convolve_ax) = plt.subplots(nrows=2, ncols=1, figsize=(6, 8))
        
        self.ax.axis([-4, 4, -5, 5])
        self.ax.grid(True)
        self.ax.axhline(0, linewidth=1, color='black')
        self.ax.axvline(0, linewidth=1, color='black')
        self.ax.set_xlabel('t')
        self.ax.xaxis.set_label_coords(1.05, 0)
        self.ax.set_title(r'Graph of g(t-$\tau$) and f($\tau$)')
        
        self.convolve_ax.axis([-4, 4, -5, 5])
        self.convolve_ax.grid(True)
        self.convolve_ax.axhline(0, linewidth=1, color='black')
        self.convolve_ax.axvline(0, linewidth=1, color='black')
        self.convolve_ax.set_xlabel('t')
        self.convolve_ax.xaxis.set_label_coords(1.05, 0)
        self.convolve_ax.set_title(r'Convolution of g($\tau$) and f($\tau$)')
        
        self.f_curve, = self.ax.plot(*self.f.generate_curve(), label=r'f($\tau$)')
        self.g_curve, = self.ax.plot(*self.g.generate_curve(True), label=r'g(t-$\tau$)')
        self.ax.legend()
        self.fill_overlap()
        
        self.c_curve, = self.convolve_ax.plot(*self.f.convolve(self.g))
        
        self.g_curve.set_marker(r"$T$")
        self.g_curve.set_markeredgecolor('r')
        self.g_curve.set_markersize(5)
        
        self.c_curve.set_marker(r"$T$")
        self.c_curve.set_markeredgecolor('r')
        self.c_curve.set_markersize(5)
        
        self.t({'new': 0})
        
        plt.close(self.fig)
        
    # Update function type
    def f_on_value_change(self, change):
        self.f = eval(change["new"])(amplitude=self.f.amplitude, width=self.f.width,
                                     delay=self.f.delay, t=self.f.t, frequency=self.f.frequency, phase=self.f.phase)
        self.f_call_back(change)
    
    # Update function type
    def g_on_value_change(self, change):
        self.g = eval(change["new"])(amplitude=self.g.amplitude, width=self.g.width,
                                     delay=self.g.delay, t=self.g.t, frequency=self.g.frequency, phase=self.g.phase)
        self.g_call_back(change)
        
    # Update function params
    def f_call_back(self, change):
        self.f.update({change["owner"].description : change["new"]})
        self.f_curve.set_ydata(self.f.generate_curve()[1])
        self.refresh()
        
    # Update function params
    def g_call_back(self, change):
        self.g.update({change["owner"].description : change["new"]})
        self.g_curve.set_ydata(self.g.generate_curve(True)[1])
        self.refresh()
        
    # Shade plot under curve
    def fill_overlap(self):
        x, f_tmp = self.f.generate_curve()
        _, g_tmp = self.g.generate_curve(True)
        z = np.zeros(len(x))
        self.ax.fill_between(x, z, f_tmp * g_tmp, color='grey', alpha='0.5')
        self.convolve_ax.fill_between(x, z, self.f.convolve(self.g)[1], color='blue', alpha='0.3')
    
    # Control shift
    def t(self, change):
        x, g_tmp = self.g.generate_curve(True)
        i = np.abs(x - change["new"]).argmin()
        self.g_curve.set_markevery(every=[i])
        self.c_curve.set_markevery(every=[i])
        self.g.update({'t':change["new"]})
        self.g_curve.set_ydata(self.g.generate_curve(True)[1])
        self.refresh()
        
    def refresh(self):
        self.ax.collections.clear()
        self.convolve_ax.collections.clear() 
        self.out.clear_output(wait=True)
        self.fill_overlap()
        self.c_curve.set_ydata(self.f.convolve(self.g)[1])
        with self.out:
            display(self.fig)
    
    def display_graph(self):
        display_list = [self.t_slider.get_display(), self.g_dropdown]
        display_list += [slide.get_display() for slide in self.g_sliders]
        display_list += [self.f_dropdown]
        display_list += [slide.get_display() for slide in self.f_sliders]
        
        display(widgets.HBox([self.out, widgets.VBox(display_list)]))
        self.refresh()


In [4]:
plt.ion()
s1 = Signal()
    
s1.display_graph()